In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF
 
config = tf.ConfigProto()  
config.gpu_options.allow_growth=True  
session = tf.Session(config=config)
 
KTF.set_session(session)

Using TensorFlow backend.


In [2]:
from Hypers import *
from Utils import *
from Preprocessing import *
from Generator import *
from Models import *

In [3]:
data_root_path = None
embedding_path = None

In [4]:
news,news_index,category_dict,subcategory_dict,word_dict,content_dict,entity_dict = read_news(data_root_path)
news_title,news_vert,news_subvert,news_entity,news_content=get_doc_input(news,news_index,category_dict,subcategory_dict,word_dict,content_dict,entity_dict)
news_info = np.concatenate([news_title,news_entity,news_vert.reshape((-1,1))],axis=-1)

In [5]:
title_word_embedding_matrix, have_word = load_matrix(embedding_path,word_dict)
entity_emb_matrix = load_entity_embedding(data_root_path,entity_dict)

In [6]:
train_session = read_train_clickhistory(news_index,data_root_path,'MINDlarge_train/behaviors.tsv')
train_user = parse_user(news_index,train_session)
train_sess, train_user_id, train_label = get_train_input(news_index,train_session)

train_generator = get_hir_train_generator(news_info,train_user['click'],train_user_id,train_sess,train_label,64)

In [7]:
test_session = read_test_clickhistory_noclk(news_index,data_root_path,'MINDlarge_test/behaviors.tsv')
test_user = parse_user(news_index,test_session)
test_docids, test_userids, test_labels, test_bound = get_test_input(news_index,test_session)

In [ ]:
model,news_encoder,inter_model, = create_model(title_word_embedding_matrix,entity_emb_matrix,category_dict)

model.fit_generator(train_generator,epochs=5)

news_scoring = news_encoder.predict(news_info,verbose=1)
test_generator = get_test_generator(news_scoring,test_docids,test_userids,test_user['click'],64)
predicted_label = inter_model.predict_generator(test_generator,verbose=1)

parse_result(predicted_label,test_bound)